## ToBigs 5주차

### Vision Advanced 과제

#### 문제 1.

Object Detection 에는 2-stage model 과 1-stage model이 존재합니다.  
각 유형에 해당하는 model을 하나씩 선정하여 설명하세요. (단, 세션 시간에 설명한 모델과 아래 제시된 모델은 제외할 것)

#### 답안 작성
2-stage 모델과 1-stage 모델은 Object Detection에서 대표적인 접근 방식입니다. 아래에서는 각각의 모델 유형에서 널리 알려진 대표적인 모델을 하나씩 설명하겠습니다.

1. 2-Stage Model: Cascade R-CNN
- 특징: Cascade R-CNN은 R-CNN 계열의 모델 중 하나로, 2-stage 구조를 개선한 모델입니다. 첫 번째 단계에서 후보 영역을 생성하고, 두 번째 단계에서는 이 후보 영역을 이용하여 더욱 정확하게 물체를 검출합니다.
- 작동 방식: 기존의 Faster R-CNN이 한 번의 ROI (Region of Interest) refinement를 거치는 것과 달리, Cascade R-CNN은 여러 단계의 refinement를 거쳐 검출 성능을 점차 향상시킵니다. 즉, 첫 번째 stage에서 Rough한 영역을 찾아낸 후, 각 stage마다 점진적으로 더 정확한 detection을 수행하는 방식입니다.
- 장점: 높은 정확도의 detection을 제공하며, 특히 다양한 크기의 물체를 검출하는 데 효과적입니다.

2. 1-Stage Model: 
- 특징: YOLO (You Only Look Once) 시리즈 중에서 최근에 발표된 YOLOv5는 1-stage detector로서 단일 신경망을 통해 이미지를 한 번에 처리하여 물체의 위치와 클래스를 예측합니다.
- 작동 방식: 입력 이미지에서 전 영역을 동시에 예측하기 때문에 매우 빠른 속도를 자랑하며, 이미지의 grid cell별로 bounding box 좌표와 클래스 확률을 예측합니다. YOLOv5는 특히 경량화와 속도 향상에 초점을 맞추어 효율적으로 물체 검출이 가능하도록 설계되었습니다.
- 장점: 실시간 검출이 가능할 정도로 빠르며, 경량화되어 있어 다양한 플랫폼에서 활용하기 좋습니다.

이렇게 Cascade R-CNN과 YOLOv5는 각각 2-stage와 1-stage 모델을 대표하는 모델로서 독특한 장점과 특성을 가지고 있습니다.

#### 문제 2.

아래 제시된 FasterRCNN 과 YOLOv5 를 각각 실행합니다.  
실행 결과를 제시하고 두 모델 사이의 차이점을 실행 결과에 근거하여 설명하세요.

Faster R-CNN은 높은 정확도를 제공하는 대신 속도가 느리고, YOLOv5는 빠른 속도로 실시간 응용에 적합하지만 정확도 측면에서는 다소 아쉬운 면이 있습니다.

In [5]:
## Package Import

import torch
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.datasets import CocoDetection
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import time
import warnings

warnings.filterwarnings("ignore")

In [ ]:
## Data Download

import os
os.makedirs('./data/', exist_ok=True)
os.makedirs('./data/images/', exist_ok=True)

!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip -d ./data/
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip -d ./data/images/

In [ ]:
## FasterRCNN

# 경로 설정
image_dir = "./data/images/val2017/"
json_path = "./data/annotations/instances_val2017.json"

# Transform 설정
transform = transforms.Compose([
    transforms.ToTensor()
])

# Dataset과 DataLoader 설정
dataset = CocoDetection(root=image_dir, annFile=json_path, transform=transform)
data_loader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=2, collate_fn=lambda x: tuple(zip(*x)))

# 모델 로드 및 설정
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# 모델의 클래스 수를 3개로 설정 (배경 포함해서 0, 2, background)
num_classes = 3  # COCO의 경우 background 포함 (0, 2 두 개의 클래스 + 배경)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# GPU 사용 가능 시 GPU로 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# 모델을 평가 모드로 설정
model.eval()

# 정확도 계산을 위한 변수
total_correct = 0
total_predictions = 0

# 소요 시간 측정 시작
start_time = time.time()

# 100장 예측 속도를 비교
cnt = 0

with torch.no_grad():
    for images, targets in data_loader:
        cnt += 1
        images = list(image.to(device) for image in images)

        # 각 이미지의 타깃을 적절히 변환하여 GPU로 전송
        processed_targets = []
        for target in targets:
            processed_target = {}
            processed_target['boxes'] = torch.tensor([ann['bbox'] for ann in target]).to(device)
            processed_target['labels'] = torch.tensor([ann['category_id'] for ann in target]).to(device)
            processed_targets.append(processed_target)

        # 모델 예측
        outputs = model(images)

        for i, output in enumerate(outputs):
            pred_labels = output['labels'].cpu().numpy()
            true_labels = processed_targets[i]['labels'].cpu().numpy()

            # 예측 수가 실제 라벨 수보다 많은 경우 예측 수를 잘라냄
            if len(pred_labels) > len(true_labels):
                pred_labels = pred_labels[:len(true_labels)]

            # 예측 레이블과 실제 레이블이 얼마나 일치하는지 확인
            correct = np.sum(pred_labels == true_labels[:len(pred_labels)])
            total_correct += correct
            total_predictions += len(true_labels)

        if cnt == 100:
            break

# 소요 시간 측정 종료
end_time = time.time()

# 소요 시간 및 정확도 출력
time_taken = end_time - start_time
accuracy = total_correct / total_predictions if total_predictions > 0 else 0

print(f"소요 시간: {time_taken:.2f}초")
print(f"정확도: {accuracy * 100:.2f}%")

In [ ]:
## YOLOv5

# 경로 설정
image_dir = "./data/images/val2017/"
json_path = "./data/annotations/instances_val2017.json"

# Transform 설정
transform = transforms.Compose([
    transforms.Resize((640, 640)),  # YOLOv5의 기본 입력 크기
    transforms.ToTensor()
])

# Dataset과 DataLoader 설정
dataset = CocoDetection(root=image_dir, annFile=json_path, transform=transform)
data_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

# 모델 로드 (PyTorch Hub에서 COCO로 사전 학습된 YOLOv5 모델 로드)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# GPU 사용 가능 시 GPU로 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval()

# 정확도 계산을 위한 변수
total_correct = 0
total_predictions = 0

# 소요 시간 측정 시작
start_time = time.time()

# 100장 예측 속도를 비교
cnt = 0

with torch.no_grad():
    for images, targets in data_loader:
        cnt += 1
        images = images.to(device)

        # 모델 예측
        outputs = model(images)

        # 예측된 바운딩 박스와 클래스 정보 추출
        for output in outputs:
            pred_labels = output[:, 5:].argmax(1).cpu().numpy()  # 예측된 클래스 레이블

            # 실제 라벨을 추출하고, 예측된 라벨과 비교
            true_labels = [t['category_id'].item() for t in targets]

            # 예측된 라벨과 실제 라벨이 일치하는지 확인
            correct = np.sum(pred_labels[:len(true_labels)] == true_labels)
            total_correct += correct
            total_predictions += len(true_labels)

        if cnt == 100:
            break

# 소요 시간 측정 종료
end_time = time.time()

# 소요 시간 및 정확도 출력
time_taken = end_time - start_time

# 정확도 계산
if total_predictions > 0:
    accuracy = (total_correct / total_predictions) * 100
else:
    accuracy = 0

# 소요 시간 및 정확도 출력
print(f"소요 시간: {time_taken:.2f}초")
print(f"정확도: {accuracy:.2f}%")